# Note:
## I assume input in format:
    -target_instrument_id = ["id1", "id2",...](list,length=dynamic,type=string)
    -target_period = [start_time, end_time](list,length=2,timestamp format = %H:%M:%S:%f",start_time<=timestamp<=end_time)
    -target_static_fields = ["field1", "field2",...](list,length=dynamic,type=string)
    -target_dynamic_fields = ["field1","field2",...](list,length=dynamic,type=string)
## Output is a pd format table:
    -instrument_id|start_time|end_time|field1|field2|...(Static)|field1|field2...(Dynamic)
    -each row represents an instrument_id
        -each cell is All Unrepeated Values Set ever shown during target_period
        -"N/A" means some instrument_id doesn't have the specific target_fields record during target_period

# 1. load data

In [1]:
import re
import pandas as pd
from datetime import datetime

## 1.1 load field_id and field_name maps(StaticFields.Txt; DynamicFields.Txt)
### -Generate dict static_fields and dynamic_fields
### -Format:{'field_name','id'}

In [2]:
def load_static_fields(file_path):
    static_fields = {}
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip empty lines or lines without the expected format
            if '\t' not in line or not line:  
                continue
            field_id, field_name = line.split('\t')
            
            # Extract the numeric part of the field ID
            field_id_numeric = re.sub(r'\D', '', field_id)  # Remove non-numeric characters using regex
            static_fields[field_name] = field_id_numeric
    
    return static_fields


In [3]:
static_fields = load_static_fields('StaticFields.Txt')
dynamic_fields = load_static_fields('DynamicFields.Txt')
print("id",static_fields.get('DTB version code'))
print("id",dynamic_fields.get('Time in minutes of the day (0..1440)'))

id 3
id 1


## 1.2 load data.txt

In [4]:
class Message:
    def __init__(self, timestamp, instrument_id, field_type, fields):
        self.timestamp = timestamp  # Timestamp of the message
        self.id = instrument_id     # Instrument ID
        self.type = field_type      # Type of the message: 'S' for static, 'D' for dynamic
        self.fields = fields        # Dictionary of field_id (without 'f') => value

    def __repr__(self):
        return f"Message(timestamp={self.timestamp}, id={self.id}, type={self.type}, fields={self.fields})"

#  parse the data from the file
def parse_data(file_path):
    messages = []
    with open(file_path, 'r') as file:
        for line in file:
            # Split the line by pipe '|' delimiter
            parts = line.strip().split('|')

            # Extract timestamp, instrument ID, and message type (S or D)
            timestamp = parts[1]
            instrument_id = parts[3]
            message_type = parts[2]  # 'S' for static fields, 'D' for dynamic fields
            
            # Fields start from index 7 onwards, and are in the form 'f<field_id>=value'
            fields = {}
            for field in parts[7:]:
                match = re.match(r'f(\d+)=(\S+)', field)
                if match:
                    field_id = match.group(1)  # Field ID without 'f' prefix
                    value = match.group(2)     # Value for that field
                    fields[field_id] = value   # Store in the dictionary
            
            # Create a Message object and add to the list
            message = Message(timestamp, instrument_id, message_type, fields)
            messages.append(message)
    
    return messages

In [5]:
messages = parse_data('data.txt')

#  2.Main function

In [6]:
def convert_to_datetime(timestamp):
    return datetime.strptime(timestamp, "%H:%M:%S:%f")

# main func
def function(target_instrument_id, target_period, target_static_fields, target_dynamic_fields, messages):
    start_time = convert_to_datetime(target_period[0])
    end_time = convert_to_datetime(target_period[1])

    rows = []  # List to store the rows for DataFrame

    # Iterate through each instrument ID
    for instrument_id in target_instrument_id:
        print("-------this is id:", instrument_id)
        static_values = {field: [] for field in target_static_fields}  # Use list to store static field values
        dynamic_values = {field: [] for field in target_dynamic_fields}  # Use list to store dynamic field values
        records_found = False
        
        # Filter messages for the current instrument_id
        instrument_messages = [message for message in messages if message.id == instrument_id]

        for message in instrument_messages:
            # Check if the timestamp is in the target period
            message_time = convert_to_datetime(message.timestamp)
            if start_time <= message_time <= end_time:
                records_found = True
                print(message)

                # for static fields (type 'S')
                if message.type == 'S':  # Static field type
                    for field_name in target_static_fields:
                        field_id = static_fields.get(field_name)
                        if f'{field_id}' in message.fields:
                            field_value = message.fields[f"{field_id}"]
                            if field_value not in static_values[field_name]:
                                static_values[field_name].append(field_value)  # Store unique values
                        else:
                            if "N/A" not in static_values[field_name]:
                                static_values[field_name].append("N/A")
                
                # for dynamic fields (type 'D')
                elif message.type !="S":  # Dynamic field type
                    for field_name in target_dynamic_fields:
                        field_id = dynamic_fields.get(field_name)
                        if f'{field_id}' in message.fields:
                            field_value = message.fields[f"{field_id}"]
                            if field_value not in dynamic_values[field_name]:
                                dynamic_values[field_name].append(field_value)  # Store unique values

        # If records were found for this instrument_id, create a row
        if records_found:
            row = {"instrument_id": instrument_id, "start_time": target_period[0], "end_time": target_period[1]}
            
            # Add static fields as individual columns with unique values as a list
            for field_name in target_static_fields:
                row[field_name] = ", ".join(map(str, static_values[field_name])) if static_values[field_name] else "N/A"
            
            # Add dynamic fields as individual columns with unique values as a list
            for field_name in target_dynamic_fields:
                row[field_name] = ", ".join(map(str, dynamic_values[field_name])) if dynamic_values[field_name] else "N/A"
            
            rows.append(row)
        else:
            # If no record is found, add an entry indicating no data
            row = {"instrument_id": instrument_id, "start_time": target_period[0], "end_time": target_period[1]}
            # Add static fields as individual columns with "N/A"
            for field_name in target_static_fields:
                row[field_name] = "N/A"
            # Add dynamic fields as individual columns with "N/A"
            for field_name in target_dynamic_fields:
                row[field_name] = "N/A"
            rows.append(row)

    # Convert to a Pandas DataFrame
    df = pd.DataFrame(rows)
        # remove remaining "N/A" if other values exist
    for column in df.columns:
        if column not in ["instrument_id", "start_time", "end_time"]:
            df[column] = df[column].apply(
                lambda x: ",".join([val for val in x.split(",") if val != "N/A"]).strip(",")
                if isinstance(x, str) and "N/A" in x and "," in x
                else x
            )


    return df

#  3.Test Samples

## 3.1 sample1

In [7]:
start_time = "15:30:00:617"
end_time = "20:00:05:525"
target_instrument_id = ["IXN22FHB34000","IXN23MKB27500","IXN22EKB61500", "IXN22BIB63500", "IXN22EKB56500", "IXN21GGB43500","IXN23MKB24000"]#
target_period = [start_time, end_time]
target_static_fields = ["Expiration day","Contract name","Source of Static (internal only)","Listing market for the exchange (US Equities)", "Root symbol (for options,...)", "Close divisor"]
target_dynamic_fields = ["Last price", "Last volume","Bid price","Bid size","Ask price","Ask size","Bid market","Ask market","Open price","High price","Open interest","Bit flag to updated last/high/low/open/vol/composite","Source of realtime message"]

df = function(target_instrument_id, target_period, target_static_fields, target_dynamic_fields, messages)

-------this is id: IXN22FHB34000
Message(timestamp=15:30:56:922, id=IXN22FHB34000, type=Q, fields={'8': '2', '4': '5.3', '5': '122', '12': '2', '6': '6.2', '7': '90'})
Message(timestamp=15:37:06:619, id=IXN22FHB34000, type=Q, fields={'8': '12', '4': '5.5', '5': '31', '12': '12', '6': '6', '7': '42'})
Message(timestamp=15:43:02:271, id=IXN22FHB34000, type=Q, fields={'8': '12', '4': '5.2', '5': '31', '12': '12', '6': '5.9', '7': '127'})
Message(timestamp=15:48:58:198, id=IXN22FHB34000, type=Q, fields={'8': '5', '4': '5.2', '5': '31', '12': '5', '6': '5.7', '7': '33'})
Message(timestamp=15:54:49:425, id=IXN22FHB34000, type=Q, fields={'8': '11', '4': '5', '5': '1', '12': '11', '6': '5.6', '7': '185'})
Message(timestamp=16:00:34:311, id=IXN22FHB34000, type=Q, fields={'8': '15', '4': '5', '5': '1', '12': '15', '6': '5.6', '7': '480'})
Message(timestamp=16:06:20:697, id=IXN22FHB34000, type=Q, fields={'8': '19', '4': '5.1', '5': '31', '12': '19', '6': '5.6', '7': '102'})
Message(timestamp=16:1

In [8]:
df

,instrument_id,start_time,end_time,Expiration day,Contract name,Source of Static (internal only),Listing market for the exchange (US Equities),"Root symbol (for options,...)",Close divisor,Last price,...,Bid size,Ask price,Ask size,Bid market,Ask market,Open price,High price,Open interest,Bit flag to updated last/high/low/open/vol/composite,Source of realtime message
0,IXN22FHB34000,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,"5, 4.8",...,"122, 31, 1, 32, 15, 21, 137, 84, 33, 10, 7, 2","6.2, 6, 5.9, 5.7, 5.6, 5.5, 5.4, 5.3, 5.2, 5.1...","90, 42, 127, 33, 185, 480, 102, 495, 149, 40, ...","2, 12, 5, 11, 15, 19, 24, 3, 1, 6, 20, 34, 36, 55","2, 12, 5, 11, 15, 19, 3, 24, 1, 6, 20, 34, 36, 55",N/A,N/A,N/A,23,FH
1,IXN23MKB27500,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,"93, 47, 187, 30, 1, 32, 83, 52, 183, 40, 131, ...","17.9, 18.2, 18.4, 18.6, 18.7, 18.9, 19, 19.1, ...","2, 20, 10, 4, 1, 30, 49, 5, 6, 71, 69, 8, 3","12, 11, 36, 3, 1, 34, 19, 14, 15, 55, 20, 5, 6","12, 11, 36, 24, 1, 3, 34, 19, 14, 15, 55, 20, ...",N/A,N/A,N/A,N/A,N/A
2,IXN22EKB61500,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,4.8,"12, 9, 8, 13, 10, 11, 15","15, 12, 34, 11, 19","15, 12, 34, 3, 11, 19",N/A,N/A,N/A,N/A,N/A
3,IXN22BIB63500,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,IXN22EKB56500,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,4.8,"11, 12, 8, 13","15, 12, 5, 34, 11, 19","15, 12, 5, 34, 11, 3, 19",N/A,N/A,N/A,N/A,N/A
5,IXN21GGB43500,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,IXN23MKB24000,15:30:00:617,20:00:05:525,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,"309, 277, 145, 39, 36, 37, 177, 197, 154, 20, ...","9.8, 10.1, 10, 10.4, 10.2, 10.3, 10.6, 10.8, 1...","20, 58, 10, 30, 3, 2, 1, 157, 6, 5, 61, 41, 37...","11, 12, 5, 14, 55, 3, 15, 6, 19, 36, 1, 24","11, 12, 5, 14, 55, 1, 3, 15, 6, 19, 36, 24",N/A,N/A,N/A,N/A,N/A
